<a href="https://colab.research.google.com/github/mark1702/mark/blob/master/Translation_Finetuning_multilang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers datasets evaluate sacrebleu
# !pip install sentencepiece
# !pip install transformers --upgrade
# !pip install accelerate -U

In [2]:
from datasets import load_dataset
books = load_dataset("opus_books", "en-fr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [3]:
books = books['train'].select(range(100)).train_test_split(test_size=0.2)

In [4]:
from transformers import AutoTokenizer

In [5]:
#!pip install sentencepiece

In [20]:
checkpoint = "facebook/hf-seamless-m4t-medium"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

In [38]:
tokenizer.src_lang='eng'
test = tokenizer('hello', max_length=128, truncation=True)
test
test_tar = tokenizer(text_target='hello', max_length=128, truncation=True)
print(test)
print(test_tar)

{'input_ids': [256047, 133863, 3, 0], 'attention_mask': [1, 1, 1, 0]}
{'input_ids': [3, 256057, 133863, 3], 'attention_mask': [1, 1, 1, 1]}


In [36]:
tokenizer.convert_ids_to_tokens(test['input_ids'])

['__kor__', '▁안녕', '하세요', '?', '▁', '</s>']

In [31]:
tokenizer.convert_ids_to_tokens(test_tar['input_ids'])

['</s>', '__fra__', '▁hello', '</s>']

In [68]:
max_input_len=128
max_target_len=128
def tokenizer_fn(batch):
    inputs = [x[source_lang] for x in batch['translation']]
    targets = [x[target_lang] for x in batch['translation']]

    tokenized_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True)
    tokenized_targets = tokenizer(text_target=targets, max_length=max_target_len, truncation=True)
    tokenized_inputs['labels'] = tokenized_targets['input_ids']
    return tokenized_inputs

In [87]:
def tokenizer_all_language_fn(batch):
    #tokenizer.src_lang=batch['translation']
    inputs = [x[source_lang] for x in batch['translation']]
    targets = [x[target_lang] for x in batch['translation']]
    tokenized_inputs = {"input_ids":[], "attention_mask":[]}
    tokenized_targets = {"input_ids":[], "attention_mask":[]}
    for i, row in enumerate(batch['translation']):
      tokenizer.src_lang = 'vie'
      tokenizer.tgt_lang = 'eng'
      input = tokenizer([row[source_lang]], max_length=max_input_len, truncation=True)
      tokenized_inputs["input_ids"].extend(input["input_ids"])
      tokenized_inputs["attention_mask"].extend(input["attention_mask"])

      target = tokenizer(text_target=[row[target_lang]], max_length=max_input_len, truncation=True)
      tokenized_targets["input_ids"].extend(target["input_ids"])
      tokenized_targets["attention_mask"].extend(target["attention_mask"])

    tokenized_inputs['labels'] = tokenized_targets['input_ids']
    return tokenized_inputs

In [88]:
tokenized_books = books.map(tokenizer_all_language_fn, batched=True, remove_columns=books["train"].column_names)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [71]:
tokenizer.vocab

{'▁ಬೇ': 30456,
 'ẹẹbu': 218742,
 '▁olduğ': 9552,
 'aribu': 82450,
 '▁δρα': 92477,
 'bel': 7312,
 'لارينين': 69567,
 '▁हुँदै': 149145,
 '▁hri': 26232,
 '▁pilie': 122815,
 '▁tshela': 58028,
 '▁य': 646,
 '▁ସେମାନଙ୍କ': 62858,
 'θυ': 65347,
 '▁Dela': 97370,
 '▁iqiniso': 105843,
 '▁іншыя': 168703,
 'やその他の': 213272,
 'илми': 218739,
 '▁לכל': 57207,
 '▁terkenal': 73380,
 '▁rekonstruk': 167996,
 '▁naujos': 240693,
 '▁بئزه': 90067,
 'මිය': 243895,
 'Ế': 255934,
 '▁ଅଲ': 133163,
 'eksp': 215480,
 'ನೊಂದಿಗೆ': 216491,
 'ருங்கள்': 247530,
 '▁رہنے': 98823,
 'ظن': 54922,
 'มาที่นี่': 225438,
 '▁ሁለቱ': 221662,
 'anes': 106936,
 'тырға': 111731,
 '▁përdorin': 222798,
 '雌': 254622,
 'image': 213807,
 '▁나': 1545,
 '▁хий': 19487,
 '▁insurance': 111679,
 '▁baraso': 134762,
 '▁продол': 45811,
 '▁מערכת': 146615,
 '▁ആസ്': 202457,
 ',19': 33123,
 '▁ತೆ': 59323,
 'ቈ': 252312,
 'zah': 53729,
 '出来る': 55835,
 '▁embereket': 179859,
 '▁말아야': 198318,
 'ସ୍ଥ': 29416,
 'salam': 190676,
 '▁സിനിമ': 106428,
 'ịtị': 64080,
 'emme

In [60]:
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 80
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 20
    })
})

In [44]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 80
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
})

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

In [19]:
model.num_parameters()

615073792

In [12]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
import evaluate
metric = evaluate.load("sacrebleu")

In [14]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [15]:
# !pip install accelerate -U

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    eval_accumulation_steps=1,
    gradient_checkpointing=True,
)

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model('checkpoint/my_saved_model')

You're using a SeamlessM4TTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
